<a href="https://colab.research.google.com/github/abuzar072/Sessions/blob/main/AI_Expense_Analyzer_practiceS2_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pandas matplotlib groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.0 MB/s eta 0:00:00


In [10]:
!pip install cloudflared



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 2.3 MB/s eta 0:00:00
  Created wheel for cloudflared: filename=cloudflared-1.0.0.2-py3-none-any.whl size=2983 sha256=249ba2d7bfea285d1b45db14083f3605ded20644b3e8e649e42364b81c8f08ea
  Stored in directory: /root/.cache/pip/wheels/5b/ec/09/c3bcd3470be046ec77a9c0cb9d8bb6ceed49c831460878ab0a
Successfully built cloudflared


In [3]:
# app.py
import os
import io
from datetime import datetime
import pandas as pd
import streamlit as st
import plotly.express as px
from groq import Groq

# ----------------------------------
# State Initialization
# ----------------------------------
def init_state():
    if "expenses" not in st.session_state:
        st.session_state.expenses = []

def expenses_df():
    df = pd.DataFrame(st.session_state.expenses)
    if df.empty:
        return pd.DataFrame(columns=["date", "amount", "category", "description"])
    df["date"] = pd.to_datetime(df["date"]).dt.date
    df["amount"] = pd.to_numeric(df["amount"])
    return df

def add_expense(date, amount, category, description):
    st.session_state.expenses.append({
        "date": str(date),
        "amount": float(amount),
        "category": category if category else "Other",
        "description": description,
    })

# ----------------------------------
# Groq LLM Integration
# ----------------------------------
def get_groq_client(api_key: str):
    return Groq(api_key=api_key)

def build_summary_text(df: pd.DataFrame) -> str:
    total = df["amount"].sum()
    by_cat = df.groupby("category", as_index=False)["amount"].sum().sort_values("amount", ascending=False)
    recent = df.sort_values("date", ascending=False).head(10)

    text = []
    text.append(f"Total Expenses: {total:.2f}\n")
    text.append("Top Categories:")
    for _, row in by_cat.iterrows():
        text.append(f"- {row['category']}: {row['amount']:.2f}")

    text.append("\nRecent Entries:")
    for _, r in recent.iterrows():
        text.append(f"- {r['date']} | {r['amount']} | {r['category']} | {r['description']}")

    return "\n".join(text)

def analyze_with_groq(df: pd.DataFrame, api_key: str):
    summary = build_summary_text(df)

    system_prompt = (
        "You are an expert personal finance AI. Analyze the spending patterns, "
        "identify problems, and provide helpful strategies to reduce expenses, "
        "optimize budgeting, and improve financial habits."
    )

    client = get_groq_client(api_key)

    try:
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": summary}
            ]
        )
        return completion.choices[0].message.content

    except Exception as e:
        return f"Groq API Error: {e}"

# ----------------------------------
# Streamlit UI
# ----------------------------------
st.set_page_config(page_title="AI Expense Analyzer", layout="wide")
st.title("💰 AI Expense Analyzer (Groq + Streamlit)")

init_state()

# Sidebar
with st.sidebar:
    st.header("Settings")
    api_key = st.text_input("Groq API Key", type="password")

    uploaded = st.file_uploader("Upload CSV (date,amount,category,description)", type=["csv"])
    if uploaded:
        try:
            df_up = pd.read_csv(uploaded)
            for _, r in df_up.iterrows():
                add_expense(r["date"], r["amount"], r["category"], r["description"])
            st.success("Expenses imported successfully.")
        except Exception as e:
            st.error(f"CSV Load Error: {e}")

    if st.button("Clear All Expenses"):
        st.session_state.expenses = []
        st.toast("All expenses cleared.")

# Main layout
col1, col2 = st.columns([2, 3])

# ----------------------------------
# Expense Input Form
# ----------------------------------
with col1:
    st.subheader("➕ Add Expense")
    with st.form("expense_form", clear_on_submit=True):
        date = st.date_input("Date", datetime.today())
        amount = st.number_input("Amount", min_value=0.0, format="%.2f")
        category = st.text_input("Category", "General")
        description = st.text_area("Description", height=60)
        submitted = st.form_submit_button("Add")

        if submitted:
            add_expense(date, amount, category, description)
            st.success("Added successfully!")

    st.markdown("---")
    st.subheader("🤖 AI Financial Analysis")

    if st.button("Analyze with Groq"):
        df = expenses_df()

        if df.empty:
            st.warning("Please add expenses first.")
        elif not api_key:
            st.error("Groq API Key required!")
        else:
            with st.spinner("Analyzing with Groq..."):
                analysis = analyze_with_groq(df, api_key)
            st.markdown("### AI Recommendations")
            st.markdown(analysis)

# ----------------------------------
# Charts & Trends
# ----------------------------------
with col2:
    st.subheader("📊 Expense Overview")
    df = expenses_df()

    st.dataframe(df, use_container_width=True)

    if not df.empty:
        # Bar Chart
        cat = df.groupby("category", as_index=False)["amount"].sum()
        fig_bar = px.bar(cat, x="category", y="amount", title="Expenses by Category")
        st.plotly_chart(fig_bar, use_container_width=True)

        # Pie Chart
        fig_pie = px.pie(cat, names="category", values="amount", title="Category Breakdown")
        st.plotly_chart(fig_pie, use_container_width=True)

        # Monthly Trend
        df["date"] = pd.to_datetime(df["date"])
        ts = df.resample("M", on="date").sum().reset_index()
        ts["month"] = ts["date"].dt.to_period("M").dt.to_timestamp()

        fig_line = px.line(ts, x="month", y="amount", markers=True,
                           title="Monthly Spending Trend")
        st.plotly_chart(fig_line, use_container_width=True)


2025-12-01 09:38:48.139 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 09:38:48.140 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 09:38:48.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 09:38:48.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 09:38:48.143 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 09:38:48.145 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 09:38:48.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 09:38:48.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [5]:
!wget -q -O - ipv4.icanhazip.com

35.229.28.168


In [11]:
!streamlit run app.py &>/content/logs.txt &


In [12]:
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


/bin/bash: line 1: cloudflared: command not found
